# CNN headline streaming
In this demo, we will scrape CNN last 50 stories html by using BeautifulSoup. The headlines containing keyword "Fast Facts" are extracted and stored locally at log dir. Spark tesxtfilestreaming check the log dir every 2 sec to do word count on the headline.   

# Web scraping 

In [4]:
import requests  
from bs4 import BeautifulSoup 
import os 
out_path=os.path.dirname(os.path.realpath('__file__'))+"/log/"
print out_path[:-1]
def webscraping(fileindex,records,url='https://www.cnn.com/specials/last-50-stories',key='Fast Fact'):
    #r=requests.get('https://www.cnn.com/specials/last-50-stories')
    r=requests.get(url) 
    soup = BeautifulSoup(r.text, 'html.parser')  
    #print soup
    results = soup.find_all('span',attrs={'class':'cd__headline-text'})

    file_name="headline%i.txt"%fileindex
    
    f = open(out_path+file_name,'a')
    for result in results:  
        headline = result.text
        if key in headline and headline not in records:
            #print headline       
            f.write('\n' + headline)
            records.append(headline)
    f.close()
    return records


/Users/yichenchang/Documents/workspace/git_submit/spark_streaming/log


## Spark text file streaming

In [113]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext


sc.stop()
# Create a local StreamingContext and batch interval of 2 second
sc = SparkContext("local","fileNetworkWordCount")
ssc = StreamingContext(sc, 2)

#read new_file from log dir
lines = ssc.textFileStream("log/")
counts = lines.flatMap(lambda line: line.split(" "))\
          .map(lambda x: (x, 1))\
          .reduceByKey(lambda a, b: a+b)
counts.pprint()

ssc.start()             # Start the computation
#ssc.awaitTermination()  # Wait for the computation to terminate

import time

i=0
records=[]
while True:
    if i>10:
        break
    else:
        #print "scraping %i"%i
        i+=1
        records=webscraping(fileindex=i,records=records)
        time.sleep(1)

-------------------------------------------
Time: 2018-01-31 07:42:26
-------------------------------------------

-------------------------------------------
Time: 2018-01-31 07:42:28
-------------------------------------------
(u'', 2)
(u'Sarah', 1)
(u'Google', 1)
(u'Guild', 1)
(u'Awards', 2)
(u'Academy', 1)
(u'Screen', 1)
(u'Fast', 6)
(u'Rico', 1)
(u'Investigation', 1)
...

-------------------------------------------
Time: 2018-01-31 07:42:30
-------------------------------------------

-------------------------------------------
Time: 2018-01-31 07:42:32
-------------------------------------------

-------------------------------------------
Time: 2018-01-31 07:42:34
-------------------------------------------

-------------------------------------------
Time: 2018-01-31 07:42:36
-------------------------------------------

-------------------------------------------
Time: 2018-01-31 07:42:38
-------------------------------------------



In [3]:
ssc.stop()
 

# Future work
1. Build structured data from web.

2. Application of Apache flume

# Reference
web scraping from http://www.dataschool.io/python-web-scraping-of-president-trumps-lies/.
